# Resultados da Survey

## Obtenção e carregamento de dados  

In [ ]:
#importação de bibliotecas
import pandas as pd

In [ ]:
# Carregar os dados principais
df_dataset = pd.read_csv("../Data/dataset.csv", encoding="utf-8", sep=";", on_bad_lines='skip')

# Carregar o mapeamento de harmonização
df_harmonization_ethinic = pd.read_csv("../Data/harmonization_ethnic_data.csv", encoding="utf-8", sep=";", on_bad_lines='skip')

## Visualização dos dados

### Visualização do dataframe principal

In [ ]:
#Exibir as primeiras linhas do dataframe
df_dataset.head()

### Visualização do dataframe de dados étnicos

In [ ]:
#Exibir as primeiras linhas do dataframe
df_harmonization_ethinic.head()

## Limpeza e Tratatamento de Dados

### Harmonização de dados étnicos

In [ ]:
# Função de harmonização com base na prioridade Bolívia > Peru > Brasil
def harmonizar_etnia(row):
    for col in ["D4_ethnic_group_bol", "D4_ethnic_group_per", "D4_ethnic_group_bra"]:
        valor = df_dataset.at[row.name, col]
        if pd.notna(valor) and valor != "":
            for origem in ["Brasil", "Peru", "Bolívia"]:
                match = df_harmonization_ethinic[df_harmonization_ethinic[origem] == valor]
                if not match.empty:
                    return match["Valor Usado"].values[0]
            return "Otro"
    return "Otro"

# Criar uma cópia do dataframe original para evitar alterações diretas
df_temp = df_dataset.copy()

# Gerar a coluna harmonizada
coluna_etnia_harmonizada = df_dataset.apply(harmonizar_etnia, axis=1)

# Inserir a coluna no local correto (antes da D4_ethnic_group_bol)
if "D4_ethnic_group_bol" in df_temp.columns:
    idx = df_temp.columns.get_loc("D4_ethnic_group_bol")
    # Inserir diretamente no ponto certo
    df_temp.insert(loc=idx, column="D4_ethnic_group", value=coluna_etnia_harmonizada)
else:
    # Caso não exista, adiciona normalmente ao final
    df_temp["D4_ethnic_group"] = coluna_etnia_harmonizada

# Remover as colunas de origem
df_temp.drop(columns=[col for col in ["D4_ethnic_group_bol", "D4_ethnic_group_per", "D4_ethnic_group_bra"] if col in df_temp.columns], inplace=True)

# Visualizar resultado
df_temp.head()


### Harmonização de estados

In [ ]:
# Criar a nova coluna D5_state com base nas colunas do df_dataset
coluna_estado_harmonizada = df_dataset["D5_state_bol"]
coluna_estado_harmonizada = coluna_estado_harmonizada.fillna(df_dataset["D5_state_per"])
coluna_estado_harmonizada = coluna_estado_harmonizada.fillna(df_dataset["D5_state_bra"])

# Inserir a nova coluna D5_state antes da D5_state_bol (se ela existir no df_temp)
if "D5_state_bol" in df_temp.columns:
    idx = df_temp.columns.get_loc("D5_state_bol")
    df_temp.insert(loc=idx, column="D5_state", value=coluna_estado_harmonizada)
else:
    df_temp["D5_state"] = coluna_estado_harmonizada

# Remover colunas originais de estado (se ainda estiverem no df_temp)
df_temp.drop(columns=[col for col in ["D5_state_bol", "D5_state_per", "D5_state_bra"] if col in df_temp.columns], inplace=True)

# Visualização opcional
df_temp.head()


### Harmonização de cidades

In [ ]:
# Criar a nova coluna D6_county com base nas colunas do df_dataset
coluna_cidade_harmonizada = df_dataset["D6_county_bol"]
coluna_cidade_harmonizada = coluna_cidade_harmonizada.fillna(df_dataset["D6_county_per"])
coluna_cidade_harmonizada = coluna_cidade_harmonizada.fillna(df_dataset["D6_county_bra"])

# Inserir a nova coluna D6_county antes da D6_county_bol (se ela existir no df_temp)
if "D6_county_bol" in df_temp.columns:
    idx = df_temp.columns.get_loc("D6_county_bol")
    df_temp.insert(loc=idx, column="D6_county", value=coluna_cidade_harmonizada)
else:
    df_temp["D6_county"] = coluna_cidade_harmonizada

# Remover colunas originais de cidade (se ainda estiverem no df_temp)
df_temp.drop(columns=[col for col in ["D6_county_bol", "D6_county_per", "D6_county_bra"] if col in df_temp.columns], inplace=True)

# Visualização opcional
df_temp.head()


## Verificação e Exportação de Dados

### Verificação

In [ ]:
#Exibir informações sobre o dataframe
df_temp.info()

#Exibir estatísticas descritivas do dataframe
df_temp.describe()


### Visualização final

In [ ]:
#Exibir as primeiras linhas do dataframe
df_temp.head()

### Exportação

In [ ]:
### Exporta o dataframe para um arquivo CSV
df_temp.to_csv('../Data/data_cleaned.csv', sep=';', encoding='utf-8-sig', index=False)